In [2]:
import requests
from bs4 import BeautifulSoup
import regex as re
from tqdm.notebook import tqdm
import pandas as pd
import time
from math import ceil

import wget

In [3]:
def get_wob_links():
    wob_links = []
    for pagenumber in tqdm(range(1, 95)):
        r = requests.get(f'https://www.utrecht.nl/bestuur-en-organisatie/publicaties/openbaar-gemaakte-informatie-na-wob-verzoeken/page/{pagenumber}/')
        soup = BeautifulSoup(r.content, 'html.parser')

        for a in soup.find(class_='limiter').find_all('a'):
            link = a['href']

            if 'besluit' in a.string:
                wob_links.append(link)
    return wob_links

wob_links = get_wob_links()

  0%|          | 0/94 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [17]:
len(wob_links)

782

In [4]:
def get_doc_type(doclink):
    """
    Based on the link to a doc, try to identify the type of document.
    """
    prefix = 'https://www.utrecht.nl/fileadmin/uploads/documenten/7.extern/wob'
    doclink = re.sub(prefix, '', doclink.lower())
    options = {'Inventaris': ['inventaris'], 'Bijlage': ['bijlage', 'document', 'agenda', 'verslag'], 'Besluit': ['besluit', 'beslissing'], 'Verzoek': ['verzoek']}
    
    for option in options:
        for keyword in options[option]:
            if keyword in doclink:
                return option

    return 'Niet herkend'

In [14]:
def read_wob(wob_link, wob_id):
    """
    Reads one wob, returns information in json format.
    Really specific for rijksoverheid.nl, because of the unique html tags and information.
    """
    source = 'https://www.utrecht.nl' 
    r = requests.get(wob_link)
    wob_meta = dict()

    # Try again in a minute if status code is not OK
    if r.status_code != 200:
        time.sleep(60)
        read_wob(wob_link, wob_id)

    soup = BeautifulSoup(r.content, 'html.parser')

    wob_meta['wob_id'] = wob_id
    wob_meta['url'] = wob_link
    # Try to find metadata of Wob-verzoek
    try:
        title = soup.find('h1').string.strip()
        wob_meta['titel'] = title
    except:
        pass
    try:
        intro = soup.find(class_='MsoNoSpacing').string.strip()
        wob_meta['beschrijving'] = intro

        try:
            date = re.search(r'\d+ [a-zA-Z]+ \d{4}', intro)
            wob_meta['datum_besluit'] = date.group()
        except:
            pass
    except:
        pass
    
    wob_meta['verantwoordelijk'] = 'Gemeente Utrecht'
    
    # Find all download links for further investigation
    download_links = [class_.find('a') for class_ in soup.find_all(class_='MsoNoSpacing') if class_.find('a')]
    documenten = []
    for id, link in enumerate(download_links):
        full_link = source + link['href']
        document = dict()
        document['document_id'] = f"{wob_id}-{id+1}"
        document['url'] = full_link
        document['bestandsnaam'] = full_link.split('/')[-1]
        document['bestandstype'] = full_link.split('/')[-1].split('.')[-1]
        document['documenttype'] = get_doc_type(full_link)
        document['titel'] = f"{wob_meta['titel']} {document['bestandsnaam'].split('.')[0]}"
        
        documenten.append(document)

    wob_meta['documenten'] = documenten
    wob_meta['aantal_documenten'] = len(documenten)
    return wob_meta

read_wob('http://utrecht.nl/bestuur-en-organisatie/publicaties/openbaar-gemaakte-informatie-na-wob-verzoeken/wob-verzoek/2021-0932-wob-besluit-betreft-schetsen-tekeningen-en-plattegronden-van-werfkelders-oudegracht/', 'UTR29')

[<a href="/fileadmin/uploads/documenten/7.extern/wob/2021/2021-0932_schetsen/Wob-verzoek.pdf">Wob-verzoek </a>, <a href="/fileadmin/uploads/documenten/7.extern/wob/2021/2021-0932_schetsen/Wob-besluit.pdf">Wob-besluit </a>]


{'wob_id': 'UTR29',
 'url': 'http://utrecht.nl/bestuur-en-organisatie/publicaties/openbaar-gemaakte-informatie-na-wob-verzoeken/wob-verzoek/2021-0932-wob-besluit-betreft-schetsen-tekeningen-en-plattegronden-van-werfkelders-oudegracht/',
 'titel': '2021-0932 Wob besluit betreft schetsen, tekeningen en plattegronden van werfkelders Oudegracht jaren negentig.',
 'beschrijving': 'Het college van burgemeester en wethouders heeft op 16 februari 2022\xa0beslist op een Wob-verzoek van een inwoner. In het Wob-verzoek wordt verzocht om verstrekking van c.q. openbaarmaking van alle schetsen, tekeningen en plattegronden die in de jaren negentig zijn gemaakt van alle kelders gelegen aan de Oudegracht in Utrecht. In onderstaande links kunt u het Wob-verzoek, Wob-besluit en de bijgevoegde documenten, die openbaar gemaakt zijn, downloaden.',
 'datum_besluit': '16 februari 2022',
 'verantwoordelijk': 'Gemeente Utrecht',
 'documenten': [{'document_id': 'UTR29-1',
   'url': 'https://www.utrecht.nl/filead

In [15]:
def read_all_wobs():
    wobs = []
    print("Getting Wob links...")
    wob_links = get_wob_links()

    print("Extracting Wob metadata...")
    for wob_id, link in tqdm(enumerate(wob_links)):
        wobs.append(read_wob(link, 'UTR' + str(wob_id)))

    return wobs

wob_meta = read_all_wobs()

Getting Wob links...


  0%|          | 0/94 [00:00<?, ?it/s]

Extracting Wob metadata...


0it [00:00, ?it/s]

[<a href="/fileadmin/uploads/documenten/7.extern/wob/2021/2021-0932_schetsen/Wob-verzoek.pdf">Wob-verzoek </a>, <a href="/fileadmin/uploads/documenten/7.extern/wob/2021/2021-0932_schetsen/Wob-besluit.pdf">Wob-besluit </a>]
[<a href="/fileadmin/uploads/documenten/7.extern/wob/2021/2021-0940_werfkelders/wob-verzoek.pdf">Wob-verzoek </a>, <a href="/fileadmin/uploads/documenten/7.extern/wob/2021/2021-0940_werfkelders/wob-besluit.pdf">Wob-besluit </a>]
[<a href="/fileadmin/uploads/documenten/7.extern/wob/2021/2021-0930_besluitvorming/20211208143531_Wob-verzoek_1.pdf">Wob-verzoek </a>, <a href="/fileadmin/uploads/documenten/7.extern/wob/2021/2021-0930_besluitvorming/20220221104457_Wob-besluit_2021-0930_ondertekend_2.pdf">Wob-besluit </a>, <a href="/fileadmin/uploads/documenten/7.extern/wob/2021/2021-0930_besluitvorming/Bijlagen.pdf">Bijlagen </a>]
[<a href="/fileadmin/uploads/documenten/7.extern/wob/2021/2021-0986_Meetrapporten/20220105134103_wob-verzoek_1.pdf">Wob-verzoek</a>, <a href="/fil

In [46]:
import json

with open('wobs_utrecht.json', 'w') as f:
    json.dump(wob_meta, f)